In [1]:
import yfinance as yf 
from database.data_manager.data_access import connect_as_user
from business_logic.get_data import fetch_portfolio
from business_logic.decision_making.strategies.SMACrossoverClass import SMACrossover
from business_logic.decision_making.data_prepration import get_OHLC_df, label_OHLC_df, split_train_test
from business_logic.models.portfolio import Portfolio
from business_logic.models.stock import Stock
from enums import Action
from datetime import datetime

In [2]:
symbol = 'AAPL'
ticker = yf.Ticker(symbol)
data = ticker.history(period='1d', interval='1m')
print('Datashape: ')
print(data.shape)
data.tail()

Datashape: 
(390, 7)


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2021-04-15 15:55:00-04:00,134.479996,134.500000,134.440002,134.490005,402931,0,0
2021-04-15 15:56:00-04:00,134.490005,134.500000,134.449997,134.455002,428859,0,0
2021-04-15 15:57:00-04:00,134.449997,134.490005,134.429993,134.429993,374250,0,0
2021-04-15 15:58:00-04:00,134.434998,134.460007,134.410004,134.419998,402721,0,0
2021-04-15 15:59:00-04:00,134.425003,134.509995,134.369995,134.509995,921270,0,0


In [3]:
data = get_OHLC_df(data)
data.tail()

,Open,High,Low,Close,Volume
Datetime,,,,,
2021-04-15 15:55:00-04:00,134.479996,134.500000,134.440002,134.490005,402931
2021-04-15 15:56:00-04:00,134.490005,134.500000,134.449997,134.455002,428859
2021-04-15 15:57:00-04:00,134.449997,134.490005,134.429993,134.429993,374250
2021-04-15 15:58:00-04:00,134.434998,134.460007,134.410004,134.419998,402721
2021-04-15 15:59:00-04:00,134.425003,134.509995,134.369995,134.509995,921270


In [4]:
decision_maker = SMACrossover()
decision_maker

<class 'business_logic.decision_making.strategies.SMACrossoverClass.SMACrossover'> with n1 = 10 and n2 = 15

In [5]:
portfolio = Portfolio()
decision_maker.n1 = 5
decision_maker.n2 = 8
for i in range(30, data.shape[0]):
    cut_data = data.iloc[(i-15) : i] 
    decision = decision_maker.should_buy(cut_data)
    time = str(cut_data.index[-1])
    print('Time: {} -- Decision: {} -- Balance: {}'.format(time, decision, portfolio.balance))

    if decision == Action.BUY:
        current_price = cut_data.Close[-1]
        stock = Stock(symbol, current_price, None, time)
        if not stock in portfolio:
            stock.set_stock_quantity(current_price, portfolio.max_per_stock)
            portfolio.add_stock(stock)
    elif decision == Action.SELL:
        stock = Stock(symbol, None, None, None)
        if stock in portfolio:
            portfolio.sell_stock(symbol)
    elif decision == Action.HOLD:
        continue

if len(portfolio.lst_stocks) > 0:
    portfolio.sell_stock(symbol)

print(portfolio.balance)

 Action.HOLD -- Balance: 4000.0
Time: 2021-04-15 11:55:00-04:00 -- Decision: Action.BUY -- Balance: 4000.0
Time: 2021-04-15 11:56:00-04:00 -- Decision: Action.HOLD -- Balance: 3328.1500244140625
Time: 2021-04-15 11:57:00-04:00 -- Decision: Action.HOLD -- Balance: 3328.1500244140625
Time: 2021-04-15 11:58:00-04:00 -- Decision: Action.HOLD -- Balance: 3328.1500244140625
Time: 2021-04-15 11:59:00-04:00 -- Decision: Action.SELL -- Balance: 3328.1500244140625
Time: 2021-04-15 12:00:00-04:00 -- Decision: Action.HOLD -- Balance: 4000.0
Time: 2021-04-15 12:01:00-04:00 -- Decision: Action.HOLD -- Balance: 4000.0
Time: 2021-04-15 12:02:00-04:00 -- Decision: Action.HOLD -- Balance: 4000.0
Time: 2021-04-15 12:03:00-04:00 -- Decision: Action.HOLD -- Balance: 4000.0
Time: 2021-04-15 12:04:00-04:00 -- Decision: Action.HOLD -- Balance: 4000.0
Time: 2021-04-15 12:05:00-04:00 -- Decision: Action.HOLD -- Balance: 4000.0
Time: 2021-04-15 12:06:00-04:00 -- Decision: Action.HOLD -- Balance: 4000.0
Time: 202

## Using ML model to make decision

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

In [9]:
data

,Open,High,Low,Close,Volume
Datetime,,,,,
2021-04-15 09:30:00-04:00,133.820007,134.250000,133.639999,134.133804,2868151
2021-04-15 09:31:00-04:00,134.157501,134.389999,134.119995,134.199997,584481
2021-04-15 09:32:00-04:00,134.199997,134.270004,134.100006,134.210007,368996
2021-04-15 09:33:00-04:00,134.220001,134.490005,134.080002,134.100006,758571
2021-04-15 09:34:00-04:00,134.009003,134.029907,134.008301,134.029907,505648
...,...,...,...,...,...
2021-04-15 15:55:00-04:00,134.479996,134.500000,134.440002,134.490005,402931
2021-04-15 15:56:00-04:00,134.490005,134.500000,134.449997,134.455002,428859
2021-04-15 15:57:00-04:00,134.449997,134.490005,134.429993,134.429993,374250
